In [40]:
import pandas as pd
from openai import OpenAI
from jinja2 import Template
from pydantic import BaseModel
from openai.lib._parsing import type_to_response_format_param
import json
from getpass import getpass

In [41]:
openai_api_key = getpass("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

# 초등 prompt 설정

In [42]:
prompt_voca_sentence_elem_template = Template("""
[PERSONA]
You are going to be given a set of English words and some of their Korean meaning. As an elementary school English teacher, generate English sentences using each of those words, and provide Korean translation, too. Overall sentences must be very easy and kids-friendly. For extra contextual reference, see the given Korean meaning.


[POLICY]
1. A sentence must not be over 15 words.
2. Separate the sentence with a tab in between.
3. For Korean translation, use '~요' at the end of each translated sentence to soften the tone. Make it natural.

[EXAMPLE]
```
Input:
    {
    "WORD" : "cat",
    "MEANING" : "고양이",
    "PART_OF_SPEECH" : "명사",
    }

Output:
    {
     "example_sentence" : "A cat is walking on the frozen lake.",
     "example_sentence_kor" : "고양이가 얼어있는 호수 위를 걷고 있어요."
    }
```                           
                           
[Input]
```
WORD : {{word}}
MEANING: {{meaning}}
PART_OF_SPEECH : {{part_of_speech}}
```                   
"""
)

- 중등

- 예시 출력

In [43]:
prompt_voca_sentence_elem = prompt_voca_sentence_elem_template.render(
    word = "everyone",
    meaning = "모든 사람",
    part_of_speech = "대명사"
)

In [44]:
print(prompt_voca_sentence_elem)


[PERSONA]
You are going to be given a set of English words and some of their Korean meaning. As an elementary school English teacher, generate English sentences using each of those words, and provide Korean translation, too. Overall sentences must be very easy and kids-friendly. For extra contextual reference, see the given Korean meaning.


[POLICY]
1. A sentence must not be over 15 words.
2. Separate the sentence with a tab in between.
3. For Korean translation, use '~요' at the end of each translated sentence to soften the tone. Make it natural.

[EXAMPLE]
```
Input:
    {
    "WORD" : "cat",
    "MEANING" : "고양이",
    "PART_OF_SPEECH" : "명사",
    }

Output:
    {
     "example_sentence" : "A cat is walking on the frozen lake.",
     "example_sentence_kor" : "고양이가 얼어있는 호수 위를 걷고 있어요."
    }
```                           
                           
[Input]
```
WORD : everyone
MEANING: 모든 사람
PART_OF_SPEECH : 대명사
```                   


In [45]:
class ExampleSentence(BaseModel):
    example_sentence : str
    example_sentence_kor : str

In [46]:
response_format = type_to_response_format_param(ExampleSentence)

In [47]:
response_format

{'type': 'json_schema',
 'json_schema': {'schema': {'properties': {'example_sentence': {'title': 'Example Sentence',
     'type': 'string'},
    'example_sentence_kor': {'title': 'Example Sentence Kor',
     'type': 'string'}},
   'required': ['example_sentence', 'example_sentence_kor'],
   'title': 'ExampleSentence',
   'type': 'object',
   'additionalProperties': False},
  'name': 'ExampleSentence',
  'strict': True}}

In [48]:
def completion(prompt : str, response_format : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort = 'low',
        messages = [
            {"role" : "system", "content" : "Generate easy example sentences."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = response_format
    )
    return response.choices[0].message.parsed

In [49]:
response = completion(prompt_voca_sentence_elem, response_format = ExampleSentence)

In [50]:
print(response)

example_sentence='Everyone is smiling in our school.' example_sentence_kor='모든 사람이 우리 학교에서 웃고 있어요.'


In [52]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent=4)
print(response_output)

{
    "example_sentence": "Everyone is smiling in our school.",
    "example_sentence_kor": "모든 사람이 우리 학교에서 웃고 있어요."
}
